In [1]:
import jieba

In [2]:
import pandas as pd
import numpy as np
from collections import Counter  

# 可能流程:
1. 訓練資料： 每個公司對應 a.一筆字串(名字/應徵內容/營業項目) b.營業額
2. 全部所有字串分詞(using jieba)
3. BoW (Bag of Words)：
4. TF-IDF 處理 a.不同筆資料但一直出現的詞 => 降低權重(ex:公司)  b.同一筆一直出現 => 增加權重(ex: 外場)
5. 初步分群看感覺(clustering)
6. 參考分群結果跟柯柯產業別，挑出幾個不同產業類代表：例如：統一便利商店是一類 / 喜來登飯店算一類...等。
7. 用 cosine 相似度判斷其他公司是上面那一類！！

# Read Data

In [3]:
company = pd.read_csv('company.csv')
print(len(company))
company.head()

604


,id,vat_number,name,address,jobs_count,g0v_data
0,7b1fce4b-5635-4a87-bb97-fee6f89280b3,17604757,飛哥文理補習班,台北市中正區衡陽路51號4樓,70,"{""error""=>true, ""message""=>""Company not found""}"
1,26455c36-97e4-42b1-871f-c651e672ee7c,50627725,陽光隔熱紙,桃園區介壽路457號,3,"{""data""=>{""地址""=>""桃園市桃園區福林里介壽路457號1樓"", ""現況""=>""核..."
2,4a53a43b-4022-4417-b2b7-c65be6fe54ef,72916919,善食堂,台北市大安區復興南路二段336號1樓,5,"{""data""=>{""財政部""=>{}}}"
3,123b5f59-56a8-4b45-bd2d-ff795c2ac350,10267365,悠菓創藝工作坊,高雄市新興區復興二路326號4樓之8,30,"{""data""=>{""地址""=>""高雄市新興區復興2路326號4樓之8"", ""現況""=>""核..."
4,f292dbb1-c0c7-497a-a3ef-5436da215f4e,12228473,神腦國際企業股份有限公司,新北市新店區中正路531號2樓,26,"{""data""=>{""財政部""=>{""行業""=>[[""272112"", ""有線電話製造""]]..."


In [4]:
# 建立一個 id 和 公司名稱的字典
id2name = {}
for i in range(len(company)):
    id2name[company['id'][i]] = company['name'][i]

In [5]:
job = pd.read_csv('job.csv')
print(len(job))
job.head()

1000


,id,title,desc,working_period,working_date,working_location,pay_wage_method,fulfil,hourly_wage,daily_wage,...,number_needed,job_candidates_count,featured,working_address,working_lat,working_lng,company_id,apply_question,administrative_area_level_3,term
0,5ddf51ad-9079-46fb-9ea0-289edc8dd579,日本展日文銷售人員,日文程度N1-N3左右工作內容主要為：協助日本師傅賣場銷售結帳翻譯日文介紹喜歡接觸人群有工作...,17：00-21：00平日17：00-22：00假日,6/26-7/8,忠孝sogo,活動結束後,False,170.0,NaN,...,6.0,8,False,106台北市大安區忠孝東路四段45號,25.041870,121.544633,43e94743-1b18-4a11-a012-2418dac3afc0,配合百貨營業時間,大安區,short_term
1,0f4243d8-77a3-4c7b-958b-11ed3cd74173,5/2 徵工讀生,桃園全大運活動協助工作人員 搬運桌椅 清點數量 桌巾 及收帆布,下午16:30-晚上20:30,2018/5/2,桃園中央大學,當日現領,True,150.0,NaN,...,20.0,33,False,NaN,NaN,NaN,7caadc54-251c-4bac-b931-f907817b224d,NaN,NaN,short_term
2,db0b76c4-69ff-4242-983e-94954719f19f,徵假日門市工讀生,1.門市客人接待，協助客人找到合適的童鞋2.門市環境清潔整理3.倉庫整理4.臨時交辦事項,早上10:00~下午18:00,2018/5/5~2019/12/31,三重,每月五日,False,150.0,NaN,...,3.0,0,False,241台灣新北市三重區捷運路19巷4弄4號,25.056412,121.484991,bfabdfc2-e0dc-4485-a4ba-19394ce020a1,NaN,三重區,long_term
3,296f2767-5abd-4f5b-9240-e17f84f8c559,徵平/假日門市工讀生,1.門市客人接待，協助客人找到合適的童鞋2.門市環境清潔整理3.倉庫整理4.臨時交辦事項,早上12:00~下午20:00,2018/5/3~2019/12/31,北投,每月五日,False,145.0,NaN,...,3.0,1,False,112台灣台北市北投區中央北路二段290號,25.138631,121.490447,bfabdfc2-e0dc-4485-a4ba-19394ce020a1,是否可配合排班,北投區,long_term
4,e6e010a6-5e24-4eb3-b373-e56781dcf096,徵外場人員、內場人員,歡迎熱情的你/妳們加入外場人員 工作內容協助點餐、帶位、送餐、收桌內場人員工作內容協助備料、...,早上11:00～下午21:00兩段班 中間休息2小時,即日起,水湳市場附近,隔月10號,False,140.0,NaN,...,4.0,1,False,406台灣台中市北屯區華美西街二段428號,24.174225,120.667934,dba152d9-3f63-4184-89d6-8581130475ed,是否可配合排班,北屯區,long_term


In [6]:
# 建立一個 id 和 求職描述
job_describe={}
for i in range(len(job)):
    job_describe[job['id'][i]] = job['desc'][i]

In [7]:
ind_word = pd.read_csv('ind_word.csv')
ind_word.columns

Index(['排除', '餐飲服務', '旅館飯店', '零售批發', '電信通訊', '物流倉儲', '資訊科技', '醫藥生物', '科技工程',
       '消費性產業', '國際貿易', '汽車製造', '財務金融', '石油製造', '休閒旅遊', '工程製造', '人力仲介', '廣告行銷',
       '媒體', '食品製造', '清潔'],
      dtype='object')

# Preparing training data

In [58]:
# 準備去除的符號和文字
symbol = ['0', '1', '2', '3', '4', '5', '6', '7', '8','9',
          '一', '二', '三', '四', '五', '六', '七', '八', '九',
          'a', 'u', 'J', 'A', 'B', 'F', 'Z', 'E', 'H', 'r', 'i', 'l','t', 'n',
          'e', 'o' ,'m', 's', 'g', 'C', 'p', 'y', 'f', 'd',
          'day', 'year', 'month','G', 'N' ,'R' ,'L', 'x',
           '"', '{', '}', '=', '>',  '\\', ' ',  '[', ']', ',', '、', 
            '）', '（', ')', '(', '.', ',', '/' ]

company_string = {} # input string


for i in range(len(company)):
    com_id = company['id'][i] # take company id
    string = company['g0v_data'][i]
    #if string.find('行業') == -1:
    #    company_string[com_id] = ''
    #    continue
        
    f1_1 = string.find('行業')+2
    f1_2 = string.find('資本額')-1
    
    if string.find('所營事業資料') == -1:
        f2_1 = string.find('營業項目')+4
    else:
        f2_1 = string.find('所營事業資料')+6
        
    f2_2 = string.find('登記機關')-1
    
    
    try:
        describe = company['name'][i]+string[f1_1:f1_2] + string[f2_1:f2_2]+ job_describe[com_id]
    except:
        describe = company['name'][i]+string[f1_1:f1_2] + string[f2_1:f2_2]
        
    for s in symbol:
        describe = describe.replace(s, '')
    company_string[com_id] = describe

In [60]:
company_string

{'7b1fce4b-5635-4a87-bb97-fee6f89280b3': '飛哥文理補習班',
 '26455c36-97e4-42b1-871f-c651e672ee7c': '陽光隔熱紙汽車零件汽車百貨批發汽機車零件配備批發業塑膠膜袋批發業建材批發業建材零售業玻璃安裝工程業汽機車零件配備零售業其他零售業室內裝潢業',
 '4a53a43b-4022-4417-b2b7-c65be6fe54ef': '善食堂財政部財政部',
 '123b5f59-56a8-4b45-bd2d-ff795c2ac350': '悠菓創藝工作坊網際網路拍賣其他服裝及其配件零售文具零售布疋衣著鞋帽傘服飾品零售業文教樂器育樂用品零售業無店面零售業I產品設計業',
 'f292dbb1-c0c7-497a-a3ef-5436da215f4e': '神腦國際企業股份有限公司有線電話製造',
 '55eda318-b504-4919-b4bc-9e31a9daed90': '鵬泰顧問有限公司電腦設備管理及資訊技術顧問資料處理網站代管及相關服務其他未分類電子零組件製造',
 'bf33fa57-4a60-4da6-9ec7-3de71b13cfad': '金哪吒休閒農場',
 'bb25e47c-3032-4315-8305-a9ff32e3f8fb': '新竹儒林',
 '8184fb39-71c6-494e-9817-b5282889848a': '康體適能其他運動及休閒教育服務運動訓練業',
 'c696d4cd-8be6-4d38-957c-c84b6e54f2ec': '玟穎貿易有限公司其他商品批發經紀其他布疋批發',
 '889b2c3b-2d3a-497d-8d38-2ccc5d7c76bb': '金田餐飲股份有限公司餐館其他管理顧問服務',
 'de493985-02cc-4580-9eca-f05ec3f665a2': '甘蔗媽媽古亭店冰果店冷熱飲店',
 '28d9e598-d60c-4d33-a1f2-83e18dbade3a': '趨勢民調台中辦公室公司名稱趨勢民意調查股份有限公司公司狀況核准設立登記機關台北市政府代表人姓名紀怡明公司所在地台北市信義區基隆路段號樓經理人名單董監事名單姓名紀怡明序號職稱董事長出資額所代表法人姓名吳世昌序號職稱董事出資額所代表法人姓名李

# Using Jieba to split the word

In [95]:
# Jieba分詞 ＆ 紀錄出現次數
counter = Counter()
for discribe in company_string.values():
    seg = jieba.cut(discribe, cut_all=False)
    token = [word for word in seg]
    #print(token)
    counter.update(token)

In [62]:
print(counter)

Counter({'零售': 583, '有限公司': 462, 'I': 436, '其他': 399, '姓名': 371, '業': 357, '服務業': 297, '公司': 282, '核准': 282, '所': 259, '法人': 255, '代表': 253, '出資額': 251, '職稱': 249, '設立': 236, '股份': 233, '日期': 217, 'h': 210, '董事': 205, '飲料': 193, '食品': 168, '批發業': 164, '經營': 164, '分公司': 164, '名單': 152, '批發': 149, '序號': 142, '及': 139, '，': 137, '或': 135, '非': 133, '得': 129, '法令': 129, '禁止': 129, '限制': 129, '之業務': 126, '管理': 123, '製': 120, '資料': 118, '名稱': 116, '顧問': 113, '狀況': 111, '台北市': 110, '所在地': 105, '店': 95, '財政部': 94, '人力': 94, '機關': 93, '用品': 90, '餐館': 89, '登記': 89, '事業': 89, '營業': 88, '除許': 86, '可業務外': 86, '店業': 82, '元': 80, '所營': 79, '代表人': 76, '號樓': 76, '董監事': 76, '資本': 76, '總額': 76, '設備': 74, '軟體': 73, '經理人': 70, '１': 69, '餐館業': 68, '什貨': 65, '造': 64, '服務': 64, '發業': 63, '電子': 63, '後': 61, '變': 60, '更': 60, '最': 59, '廣告': 57, '之': 57, '電腦': 56, '０': 56, '政府': 54, '分類': 54, '資訊': 53, '顧問業': 53, '品': 53, '地址': 52, '工程': 51, '未分類': 50, '出版': 50, '長': 48, '企業': 47, '文教': 46, '監察人': 46, '未': 46, '

In [73]:
useless = ['台灣', '台北', '台北市', '北市', '桃園', '大安','中壢區',  '敦化北路', '敦北', '中山' 
           '中正路', '市桃園區','高雄', '高雄市', '前鎮區', '新莊', '帕里', '新北市', '莊區', 
           '新泰',  '前鎮', '新竹市','新竹','中正', '內湖', '成功', '中國','南京','中山',
           '北屯', '中市', '桃園市', '松山' ,'明湖', '東路', '湖區', '信義區','新北',
           
           '民意', '登記', '機關', '政府', '代表人',  '經理人', '名單', '董監事', '職稱', '董事', 
           '出資額', '代表', '法人', '序號', '監察人', '資本', '總額', '所營', '事業', 
           '其他', '及其', '未分類', '之業務', '日期', '設立', 'yeamoh', 'yemoh','dyyemoh',
            '除許', '核准', '除外', '有限公司', '業除','業國際','富邦','財產','類型',
           '不得', '法令', '禁止', '限制','可業務外','供應','公室','名單', 
            '骨拉', '玄惠',  '指導委員會', '發國產', '林明宏', '服務','內設',
          '突出', '喬荃', '一樓', '有限', '責任', '名稱',  '所在地', '市政府',
           '\u3000', '姓名', '\xa0', '業無','產品','冷熱','商品','組織',
           '股份', '處理', '相關', '發業',  '店業', '綜合', '蔬菜', '黎明', '藝珂', 
           '人事', '分公司', '資訊供', '行業', '資本額', '營業',  '財政部','行業',
           '資本額','營業','地址','人名','稱藝','負責人','姓氏','使用','統一','發票',
           '總機','構統','編號','狀況','路段','號樓','經理','玉芬','總本','公司', 
            '青年', '業受', '花卉', '場所',  '統發票','經濟部','現況','項目',
           '統編號', '號設立', '公開',  '富商',  '聚鍋', '衡陽', '分類', '業化',  
           '業布', '金螺絲', '螺帽', '鉚釘', '許可業務外','路號', '週邊', '不動', 
           '料及', '水產品', '中心', '北市', '地下', '資本額', '貴金屬','農林', '漁礦']

In [74]:
# 給詞一個編號 & 只出現一次的去掉 & 只有一個字的去掉
word2idx = {}
idx2word = {}
num_words = 0
for word, count in counter.items():
    if count <= 10 or len(word)==1:
        continue
    elif word in useless:
        continue
        
    word2idx[word] = num_words
    idx2word[num_words] = word
    num_words += 1

In [75]:
print('vector length: ', len(word2idx))
print('word index:\n', word2idx)

vector length:  141
word index:
 {'文理': 0, '補習': 1, '汽車': 2, '零件': 3, '配備': 4, '批發業': 5, '建材': 6, '零售': 7, '安裝': 8, '工程': 9, '網路': 10, '服裝': 11, '配件': 12, '衣著': 13, '鞋帽': 14, '傘服': 15, '飾品': 16, '文教': 17, '樂器育樂': 18, '用品': 19, '店面': 20, '設計業': 21, '國際': 22, '企業': 23, '電腦': 24, '設備': 25, '管理': 26, '資訊': 27, '技術': 28, '顧問': 29, '資料': 30, '電子': 31, '休閒': 32, '運動': 33, '批發': 34, '經紀': 35, '餐館': 36, '冰果': 37, '飲店': 38, '智慧': 39, '權業': 40, '顧問業': 41, '問業': 42, '服務業': 43, '軟體': 44, '般廣告': 45, '廣告': 46, '傳單': 47, '分送': 48, '市場': 49, '研究': 50, '調查業': 51, '藝文': 52, '會議': 53, '展覽': 54, '工商': 55, '業圖': 56, '出版': 57, '經營': 58, '商行': 59, '餐廳': 60, '食品': 61, '什貨批': 62, '什貨': 63, '飲料': 64, '貿易業': 65, '餐館業': 66, '電視': 67, '玩具': 68, '娛樂': 69, '投資': 70, '科技股份': 71, '代理': 72, '仲介': 73, '雜誌': 74, '整合': 75, '行銷': 76, '傘服飾': 77, '品批': 78, '演藝': 79, '活動業': 80, '館業': 81, '表演': 82, '租賃業': 83, '計業': 84, '人力': 85, '派遣': 86, '電器': 87, '安裝業': 88, '短期': 89, '科技': 90, '材料': 91, '事務性': 92, '機器': 93, '服飾品': 94, '活動': 9

# Bag of word

In [86]:
def weight(word):
    '''特殊字增加權重'''
    im1 = ['零售', '批發業', '什貨','批發']
    im2 = ['金融']
    im3 = ['便利商店', '合作社']
    im4 = ['旅館', '飯店', '酒店', '館業']
    im5 = ['軟體', '電腦', '數位','科技股份','電子']
    im6 = ['補習', '教育', '文教', '文理', '補習'] 
    im7 = ['餐食', '餐館', '餐食', '餐飲業', '料理', '餐食', '餐館業', '速食店', '小吃店']
    im8 = ['經營業', '顧問', '顧問業',  '投資', '管理',  '設計業', '經紀','指導', '經營']
    im9 = ['冰果', '店冰果', '飲店', '飲料', '咖啡', ] 
    im10 = ['紡織', '工程', '技術', '企業社', '零件''機械',]
            
    if word in im1:
        w = 10
    elif word in im2:
        w = 20
    elif word in im3:
        w = 100
    elif word in im4:
        w = 100
    elif word in im5:
        w = 15
    elif word in im6:
        w = -100
    elif word in im7:
        w = 100
    elif word in im8:
        w = 10
    elif word in im9:
        w = 30
    elif word in im10:
        w = 50
        
    else:
        w = 1
        
    return w
    
    
    
    
    
    

In [87]:
BoW = {}
idx2name = {}

i = 0
train_data = np.zeros((len(company), len(word2idx)))
for com_id, describe in company_string.items():
    vector = np.zeros(len(word2idx))
    
    for word, idx in word2idx.items():
        freq = describe.count(word)
        weig = weight(word)

            
        vector[idx] = freq*weig
    BoW[id2name[com_id]] = vector
    train_data[i,:] = vector
    idx2name[i] = id2name[com_id]
    i += 1
    

# 分群

In [88]:
# from sklearn.metrics.pairwise import cosine_similarity 
# cosine_similarity(train_data)

In [89]:
from sklearn.cluster import KMeans

In [90]:
km = KMeans(n_clusters=10, init='k-means++', max_iter=100, n_init=1)

In [91]:
y_pred = km.fit_predict(train_data)

In [92]:
clustering = pd.DataFrame()
c = []
n = []

for k in range(10):
    for i in range(y_pred.shape[0]):
        if y_pred[i] == k:
            c.append(k)
            n.append(idx2name[i])

            
clustering['class'] = c
clustering['name']= n
        

clustering.to_csv('clustering.csv', sep=',', header=True, index=False)

In [93]:
y_pred

array([8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 2, 0, 0, 0, 0, 8, 0,
       8, 3, 0, 8, 0, 8, 0, 0, 0, 8, 0, 0, 0, 4, 8, 0, 8, 0, 0, 0, 0, 0,
       0, 8, 0, 2, 4, 8, 0, 7, 0, 0, 0, 2, 0, 9, 3, 0, 3, 0, 0, 6, 0, 4,
       0, 8, 0, 3, 0, 0, 8, 0, 7, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 3, 0, 0,
       7, 0, 5, 0, 7, 0, 0, 7, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 4, 9,
       8, 0, 8, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 8, 3, 0, 7, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       4, 0, 0, 0, 0, 0, 0, 2, 2, 0, 3, 0, 0, 0, 0, 7, 0, 6, 0, 7, 9, 0,
       0, 0, 0, 8, 8, 3, 0, 3, 0, 0, 8, 0, 0, 7, 0, 0, 0, 1, 8, 7, 0, 0,
       6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       7, 0, 2, 2, 0, 0, 7, 4, 0, 0, 0, 7, 0, 6, 0, 0, 0, 0, 4, 0, 0, 1,
       0, 0, 2, 0, 2, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 4, 9, 0, 2, 0, 0, 7,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 1,

In [ ]:
# # Jieba 測試： 效果還可以
# text = '葳柏貿易(深圳)有限公司。木薯澱粉和冷凍牛肉的進口與銷售，國外採購人員'
# seg = jieba.cut(text, cut_all=False)
# print("/".join(seg))